# Choice of Map
In a first step, a suitable data set was downloaded from this [link](https://www.openstreetmap.org/export#map=13/54.7834/9.4333), using the Overpass API.

The data set contains [Open Street Map](http://www.openstreetmap.org) data from the city of Flensburg located in northern Germany.

# Auditing and Cleaning
The data set was audited and cleaned in several steps. First, a sample of the data set as created using the [sample script](sample.py), provided by Udacity. This yielded a much smaller file (6872kB) as compared to the original uncompressed OSM file (68845kB). Only when all cleaning and auditing scripts could be run successfully on this sample data, it was actually run on the full data set.

When auditing the data set, a number of problems where encountered, namely:
* different writings of the same street names
* handling of unicode characters in Python 2.7
* invalid postal codes
* inconsistent formatting of phone numbers

These problems and inconsistencies where determined using an [auditing script](audit.py) in plain Python 2.7, and cleaned using the [.csv export](osm2csv.py) script.

The cleaning functions used during both auditing and exporting can be found in the cell below.

Typical errors, when dealing with german street names, are inconsistent abbreviation for the word "Straße" (street). These should have been cleaned automatically. However, the contributors to the data set under consideration seem to have done a remarkable job in keeping naming conventions. All occurrences of the word "Straße" in the data set where written in the same way. However, after checking only slight differences in street names in the [auditing script](audit.py), it was found, that there where indeed a few streets with inconsistent or wrong names. These were cleaned "manually" in the cleaning script, as shown below.

In [1]:
import string
import re

def clean_street(street, handcleaning):
    if street.lower().rfind('strasse') != -1: # convert to lower and look for 'strasse'
        return string.replace(street,'trasse','traße') # return street with 'trasse' replaced by 'traße' in order to preserve capitals
    elif street.lower().rfind('str.') != -1: # not found, then look for 'str.'
        return string.replace(street,'str.','straße') # return street with 'str.' replaced by 'straße'
    
    # hand cleaned street names
    if handcleaning:
        if street.lower().rfind('schloss') != -1: # find Schloss/Schloß (castle), which also has the problematic letter ß
            return string.replace(street,'chloss','chloß') # replace 'ss' with 'ß'    
        if street == 'Scandinavian Park' or street == 'Scandinavien-Park':
            return 'Scandinavian-Park'
        if street == 'Geheimrat-Dr.-Schaedel-Straße':
            return 'Geheimrat-Doktor-Schaedel-Straße'
    
    return street

def clean_postal_code(postal_code):
    postal_code_area = re.compile(r'^2[0-5]{1}[0-9]{3}') # regex for postal codes in northern Germany
    if postal_code_area.search(postal_code):
        return postal_code
    else:
        return u'no valid postal code'

def clean_phone(phone):    
    # convert phone numbers according to recommendation E.123 from ITU
    # https://en.wikipedia.org/wiki/E.123
     
    phone = phone.translate(None,'/-(). ')             # remove all special characters commonly used in phone numbers, except +
    if phone.startswith('00'):                    # replace leading 00 from country code with +
        phone = phone.replace('00','+',1)
    elif phone.startswith('0'):                    # replace leading 0 from area code with '+49 '
        phone = phone.replace('0','+49',1)      
    
    l = list(phone)
    if phone.startswith('+'):                    # add whitespace after two-digit country code and again after 3-digit area code (only for Germany)
        l.insert(3, ' ')
        if phone.startswith('+49'):
            l.insert(7, ' ')
        phone = ''.join(l)
    
    return phone

# Create SQlite DB and Import Data from .csv Files
After cleaning and exporting the osm data set to .csv files, these files where inserted in a newly created sqlite3 data base as described below:

Create sqlite db using the example schema.sql from a windows command line:

```
sqlite3 osm_project.db < schema.sql
```
Open data base
```
sqlite osm_project.db
```

Import previously created .csv files

```
sqlite> .mode csv
sqlite> .import nodes.csv nodes
sqlite> .import nodes_tags.csv nodes_tags
sqlite> .import ways.csv ways
sqlite> .import ways_nodes.csv ways_nodes
sqlite> .import ways_tags.csv ways_tags
```

In a first attempt to import the .csv files, I got the following fault message:
> INSERT failed: datatype mismatch

Changing the line ```codecs.open(NODES_PATH, 'w')``` to ```codecs.open(NODES_PATH, 'wb')``` in the [.csv export script](osm2csv.py) to force binary and suppress extra carriage returns in the .csv-files solved the problem.

# Overview of the Data Base
In order to get an overview of the structure of the data base, all tables are queried below and the first few lines are displayed.

In [2]:
import sqlite3
import pandas as pd

# Fetch records from osm_project.db
db = sqlite3.connect("osm_project.db")
c = db.cursor()

In [3]:
QUERY = "SELECT * FROM nodes;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['id','lat','lon','user','uid','version','changeset','timestamp'])
df.head()

,id,lat,lon,user,uid,version,changeset,timestamp
0,496065,54.772389,9.343453,simlox,795290,7,57804251,2018-04-04T13:44:12Z
1,496066,54.772097,9.344723,simlox,795290,6,57804251,2018-04-04T13:44:12Z
2,496067,54.771689,9.345011,amd64-online,579833,4,57255314,2018-03-16T22:44:50Z
3,496068,54.771262,9.345849,Peilscheibe,35560,3,3486195,2009-12-29T23:08:24Z
4,496069,54.771235,9.346165,amd64-online,579833,4,57255314,2018-03-16T22:44:50Z


In [4]:
QUERY = "SELECT * FROM nodes_tags;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['id','key','value','type'])
df.head()

,id,key,value,type
0,496722,cid_58:tabcd_1:Class,Point,TMC
1,496722,cid_58:tabcd_1:Direction,positive,TMC
2,496722,cid_58:tabcd_1:LCLversion,9.00,TMC
3,496722,cid_58:tabcd_1:LocationCode,44623,TMC
4,496722,cid_58:tabcd_1:NextLocationCode,44624,TMC


In [5]:
QUERY = "SELECT * FROM ways;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['id','user','uid','version','changeset','timestamp'])
df.head()

,id,user,uid,version,changeset,timestamp
0,2557562,Kent Friis,145774,20,18957966,2013-11-17T18:42:57Z
1,2557564,Maarten Deen,9176,19,38933548,2016-04-27T20:32:00Z
2,2862595,MKnight,39774,17,45913518,2017-02-08T11:25:24Z
3,4006063,simlox,795290,18,57804251,2018-04-04T13:44:20Z
4,4586893,MKnight,39774,15,45913518,2017-02-08T11:25:25Z


In [6]:
QUERY = "SELECT * FROM ways_tags;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['id','key','value','type'])
df.head()

,id,key,value,type
0,2557562,left,track,cycleway
1,2557562,highway,tertiary,cycleway
2,2557562,name,Padborgvej,cycleway
3,2557562,surface,asphalt,cycleway
4,2557564,cycleway,lane,regular


In [7]:
QUERY = "SELECT * FROM ways_nodes;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['id','node_id','position'])
df.head()

,id,node_id,position
0,2557562,441633170,0
1,2557562,11307072,1
2,2557562,420322536,2
3,2557562,499421438,3
4,2557562,11307073,4


Determine number of distinct contributors, number of nodes and number of ways

In [8]:
QUERY = "SELECT count() FROM (SELECT DISTINCT user FROM nodes);"
c.execute(QUERY)
users_no = c.fetchone()
print 'Number of distinct contributors: ' + str(users_no[0])

QUERY = "SELECT count() FROM nodes;"
c.execute(QUERY)
nodes_no = c.fetchone()
print 'Number of nodes in the data set: ' + str(nodes_no[0])

QUERY = "SELECT count() FROM ways;"
c.execute(QUERY)
ways_no = c.fetchone()
print 'Number of ways in the data set: ' + str(ways_no[0])

Number of distinct contributors: 422
Number of nodes in the data set: 283262
Number of ways in the data set: 55358


# Additional Statistics
Determine, which users contributed most

In [9]:
QUERY = "SELECT user,count() FROM nodes GROUP BY user ORDER BY count() DESC;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['user', 'count'])
df.head()

,user,count
0,Zartbitter,219739
1,Vasco_da_Gama,8168
2,Hubert87,4904
3,Hjart,4097
4,Kent Friis,2931


We have a clear winner: Zartbitter contributed the most by a big margin! Thinking about it, it seems a bit strange, so let's take a look at the time span in which Zartbitter has made so many entries:

In [10]:
QUERY = "SELECT min(timestamp), max(timestamp) FROM nodes WHERE user='Zartbitter';"
c.execute(QUERY)
print 'Timespan Zartbitter: ' + str(c.fetchall())

Timespan Zartbitter: [(u'2008-06-08T09:45:07Z', u'2018-04-15T11:06:38Z')]


That looks like an awful lot of entries for Zartbitter. Actually in approximately 10 years, the user would have had to make approximately 220000/10/365, that's 60 entries per day! There must be something wrong:

In [11]:
QUERY = "SELECT * FROM nodes WHERE user='Zartbitter' ORDER BY timestamp DESC LIMIT 100;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['id','lat','lon','user','uid','version','changeset','timestamp'])
df.head()

,id,lat,lon,user,uid,version,changeset,timestamp
0,5550668623,54.808580,9.364449,Zartbitter,40397,2,58107568,2018-04-15T11:06:38Z
1,5550668818,54.808644,9.363458,Zartbitter,40397,2,58107568,2018-04-15T11:06:38Z
2,5550789861,54.815002,9.362418,Zartbitter,40397,2,58107568,2018-04-15T11:06:38Z
3,5550789862,54.815146,9.362917,Zartbitter,40397,2,58107568,2018-04-15T11:06:38Z
4,5550789863,54.814919,9.362471,Zartbitter,40397,2,58107568,2018-04-15T11:06:38Z


In [12]:
QUERY = "SELECT * FROM nodes WHERE user='Vasco_da_Gama' ORDER BY timestamp DESC LIMIT 100;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['id','lat','lon','user','uid','version','changeset','timestamp'])
df.head()

,id,lat,lon,user,uid,version,changeset,timestamp
0,1078973812,54.748988,9.529197,Vasco_da_Gama,251118,4,48471567,2017-05-07T11:04:26Z
1,1078973926,54.749503,9.529000,Vasco_da_Gama,251118,3,48471567,2017-05-07T11:04:26Z
2,1078974119,54.749446,9.527991,Vasco_da_Gama,251118,3,48471567,2017-05-07T11:04:26Z
3,4839727289,54.749413,9.528052,Vasco_da_Gama,251118,1,48471567,2017-05-07T11:04:26Z
4,4839727290,54.749452,9.528785,Vasco_da_Gama,251118,1,48471567,2017-05-07T11:04:26Z


There seem to be a lot of versions for each of the entries by Zartbitter, when comparing to Vasco_da_Gama's entries. However, if this is already suspicious, regarding validity of the data can't be assessed at this point. A possible way to handle this, would be to group by changeset before counting entries in order to count only real entries.

In [13]:
QUERY = "SELECT user,count() FROM (SELECT * FROM nodes GROUP BY changeset)  GROUP BY user ORDER BY count() DESC;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['user', 'count'])
df.head()

,user,count
0,Zartbitter,1716
1,Vasco_da_Gama,280
2,bjay999,183
3,OSM-Sven,160
4,alfa91,96


This looks much more reasonable!

In order to find a few more meaningful insights regarding the data set, all keys available in the "ways_tags" table of the data set were determined.

In [14]:
QUERY = "SELECT DISTINCT key,value FROM ways_tags GROUP BY key ORDER BY key;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['key','value'])
df[160:165]

,key,value
160,maxheight,none
161,maxspeed,30
162,maxspeed:backward,sign
163,maxstay,2h
164,maxweight,5.5


An interesting aspect found, was the distribution of speed limits in the city of Flensburg. There are actually more streets with a speed limit of only 30 km/h than there are with a speed limit of 50 km/h, which is the normal speed limit within city boundaries in Germany.

In [15]:
QUERY = "SELECT key,value, count() FROM ways_tags WHERE key='maxspeed' GROUP BY value ORDER BY count() DESC;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['key','value','count'])
df.head()

,key,value,count
0,maxspeed,30,567
1,maxspeed,50,492
2,maxspeed,DE:urban,113
3,maxspeed,sign,85
4,maxspeed,100,83


As Flensburg seems to be a low speed city, wouldn't it be interesting ho is the person adding most cycleways to the open street map? This will use a query over several tables as well...

In [16]:
QUERY = "SELECT user, count() FROM ways JOIN ways_tags ON ways.id=ways_tags.id WHERE type='cycleway' GROUP BY user ORDER BY count() DESC;"
c.execute(QUERY)
rows = c.fetchall()
  
df = pd.DataFrame(rows, columns=['user','count'])
df.head()

,user,count
0,Fördenixe,88
1,Kent Friis,14
2,Tapsiturtle,11
3,f_l_b123,7
4,Henrik PS,6


It seems Fördenixe is a pretty active bicycle rider :-)

# Summary and Additional Ideas
An open street map data set was downloaded and prepared for investigation using Python 2.7, SQlite3 and Jupyter Notebooks. The city under investigation was Flensburg. The file size of the unsuppressed data set was approximately 68MB.

A number of data auditing and cleaning operations have been carried out on the data sets, before extracting a number of statistics regarding the data. Faulty and inconsistent street names as well as invalid postal codes and inconsistent phone number formats where rectified, before exporting the data to .csv-files, which where then imported in an sqlite3 data base.

Validity and uniformity of the data was checked during the import of the .csv files using the predefined schema.

It was found that for some statistics, like the number of contributions of a certain user, care needs to be taken when grouping entries in order to get meaningful and accurate results.

The following references are made between tables in the sqlite data base:

> nodes_tags.id REFERENCES nodes.id

> ways_tags.id REFERENCES ways.id

> ways_nodes.id REFERENCES ways.id

> ways_nodes.node_id REFERENCES nodes.id

Additional checks regarding completeness and consistency of these references should be carried out on the data sets, in order to increase confidence in the results.

If broken references could be found, for example using a statement like in the following cell, it has to be decided, if and how these missing references should be treated in order to least affect statistics. In any case, missing references will decrease confidence in the accuracy of the results.

In [17]:
QUERY = "SELECT count() FROM nodes_tags WHERE nodes_tags.id NOT IN (SELECT id FROM nodes WHERE nodes.id = nodes_tags.id);"
c.execute(QUERY)
print 'Missing references in nodes_tags: ' + str(c.fetchone()[0])

Missing references in nodes_tags: 0


In [18]:
db.close()